In [388]:
from astropy.io import fits
from astropy.nddata import CCDData
from astropy.table import Table
from astropy.nddata import NDDataArray, StdDevUncertainty
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import collections
%matplotlib inline

# Mathematical Steps
For a given spatial pixel $p = (x,y)$ with $N$ values of observed line ratios $R_N(p)$ with errors on the line ratios $E_N(p)$ in same units as $R_N$, we use predicted line ratios as function of $A_N(G_0,n)$, to compute differences $\delta_N$ across all $A$:
$${\delta_N^2(p,G_0,n)} = {\left[ R_N(p) - A_N(G_0,n)\over E_N(x)\right] }^2$$
 
The reduced chi-squared for spatial pixel $i$ is then given by

$$\chi^2(p,G_0,n) =  \sum_N {\delta_N^2(p,G_0,n) \over N-1}$$

and the best-fit value of $(G_0,n)$ is the minimum of $\chi^2$  The range of $(G_0,n)$ over which $A_N$ have been computed must be large enough to encompass physically reasonable values in the ISM, typically $10^{-1} < G_0 < 10^7$ in Habing units and $1 < n < 10^7$ cm$^{-3}$

For a spatial map, we do the above computation over all $p$ to produce a spatial maps of $\chi^2$ and best-fit spatial maps of $(G_0,n)$.

# Logical Steps
Given a list of measurement IDs,
1. Find all available matching model ratios
2. For each matching model ratio, compute the corresponding measurement ratios (including error propagation)
3. For each measurement ratio, compute the $\delta^2$ as above, using the ratio label as in index to find the corresponding FITS file of $A(G_0,n)$.
4. Compute $\chi^2$ from the $\delta$'s as above.
    

In [401]:
# maybe this is just better storing this info in a Table.  Then adding models is 
# just adding to the table
class ModelRatioData:
    def __init__(self,numerator, denominator, identifier,filename,metallicity=1):
        self._numerator = numerator
        self._denominator =denominator
        self._identifier = identifier
        self._filename = filename + "web"
        self._metallicity= metallicity
        
    @property
    def id(self): return self._identifier

    @property 
    def filename(self): return self._filename
    
    @property 
    def metallicity(self): return self._metallicity
    
    @property
    def isSolarMetallicity(self):
        return self._metallicity == 1

class Measurement(NDDataArray):
    def __init__(self,*args, **kwargs):
        self._identifier = kwargs.pop('identifier', None)
        super().__init__(*args, **kwargs)
        #print(self._uncertainty.parent_nddata)
        #.parent_nddata(self._data)
        
        
    @property
    def flux(self):
        return self.data
    
    @property
    def id(self):
        return self._identifier
    
    @property
    def error(self):
        return self.uncertainty
    
    
    def add(self,other):
        z=super().add(other)
        z._identifier = self.id + '+' + other.id
        return z
   
    def subtract(self,other):
        z=super().subtract(other)
        z._identifier = self.id + '-' + other.id
        return z
    
    def multiply(self,other):
        #zz = super()
        #print(type(zz))
        z=super().multiply(other)
        #print(self.id,other.id)
        z._identifier = self.id + '*' + other.id
        return z
        
    def divide(self,other):
        #zz = super()
        #print(type(zz))
        z=super().divide(other)
        #print(self.id,other.id)
        z._identifier = self.id + '/' + other.id
        return z
    
    def __add__(self,other):
        z=self.add(other)
        return z
    def __sub__(self,other):      
        z=self.subtract(other)
        return z
    
    def __mul__(self,other):
        z=self.multiply(other)
        return z
    
    def __truediv__(self,other):
        z=self.divide(other)
        return z
    
class PDRutils:
    def __init__(self,models):
        if type(models) == str:
            self.initializeFromFile(models)
        else:
            self._models = models
        
    def initializeFromFile(self,filename):
        self._models=Table.read(filename,format="ascii.ipac")
        self._models.add_index("label")
    
   
    def find_ratio_elements(self,m):
        """Return an iterator of valid numerator,denominator pairs for the given list of measurement IDs"""
        if not isinstance(m, collections.abc.Iterable) or isinstance(m, (str, bytes)) :
            raise Exception("m must be an array of strings")
            
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            z = dict()
            if s in self._models["label"]:
                z={"numerator":self._models.loc[s]["numerator"],
                   "denominator":self._models.loc[s]["denominator"]}
                yield(z)
                
    def get_ratio_elements(self,m):        
        if not isinstance(m, collections.abc.Iterable) or isinstance(m, (str, bytes)) :
            raise Exception("m must be an array of strings")
        k = list()   
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                z={"numerator":self._models.loc[s]["numerator"],
                   "denominator":self._models.loc[s]["denominator"]}
                k.append(z)
                
        return k
        
            
    def find_pairs(self,m):
        """Return an iterator of model ratios labels for the given list of measurement IDs"""
        if not isinstance(m, collections.abc.Iterable) or isinstance(m, (str, bytes)) :
            raise Exception("m must be an array of strings")
            
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                yield(s)
    
    def find_files(self,m,ext="fits"):
        """Return an iterator of model ratio files for the given list of measurement IDs"""
        if not isinstance(m, collections.abc.Iterable) or isinstance(m, (str, bytes)):
            raise Exception("m must be an array of strings")
        for q in itertools.product(m,m):
            s = q[0]+"/"+q[1]
            if s in self._models["label"]:
                yield(self._models.loc[s]["filename"]+"."+ext)
    
    def ratiocount(self,m):
        """Return the number of model ratios found for the given list of measurement IDs"""
        # since find_files is a generator, we can't use len(), so do this sum.
        # See https://stackoverflow.com/questions/393053/length-of-generator-output
        return(sum(1 for _ in self.find_files(m)))
                
    def read_fits(self,m):
        d = "/n/lupus/mpound/WITS/Docs/pdrt/"
        self._fitsfiles = dict()
        for p in self.find_files(m):
            self._fitsfiles[p] = fits.open(d+p)
            
    def read_ccd(self,m,unit):
        d = "/n/lupus/mpound/WITS/Docs/pdrt/"
        self._fitsfiles = dict()
        for p in self.find_files(m):
            self._fitsfiles[p] = CCDData.read(d+p,unit=unit)
    
    
                

In [390]:
ratiodict = {
    "OI_145/OI_63"   : "oioi",
    "OI_145/CII_158" : "o145cii",
    "OI_63/CII_158"  : "oicp",
    "CII_158/CI_609" : "ciici609",
    "CI_370/CI_609"  : "cici",
    "CII_158/CO_10"  : "ciico",
    "CI_609/CO_10"   : "cico",
    "CI_609/CO_21"   : "cico21",
    "CI_609/CO_32"   : "cico32",
    "CI_609/CO_43"   : "cico43",
    "CI_609/CO_54"   : "cico54",
    "CI_609/CO_65"   : "cico65",
    "CO_21/CO_10"    : "co2110",
    "CO_32/CO_10"    : "co3210",
    "CO_32/CO_21"    : "co3221",
    "CO_43/CO_21"    : "co4321",
    "CO_65/CO_10"    : "co6510",
    "CO_65/CO_21"    : "co6521",
    "CO_65/CO_54"    : "co6554",
    "CO_76/CO_10"    : "co7610",
    "CO_76/CO_21"    : "co7621",
    "CO_76/CO_43"    : "co7643",
    "CO_76/CO_54"    : "co7654",
    "CO_76/CO_65"    : "co7665",
    "CO_87/CO_54"   : "co8754",
    "CO_87/CO_65"   : "co8765",
    "CO_98/CO_54"   : "co9854",
    "CO_98/CO_65"   : "co9865",
    "CO_109/CO_54"   : "co10954",
    "CO_109/CO_65"   : "co10965",
    "CO_1110/CO_54"   : "co111054",
    "CO_1110/CO_65"   : "co111065",
    "CO_1211/CO_54"   : "co121154",
    "CO_1211/CO_65"   : "co121165",
    "CO_1312/CO_54"   : "co131254",
    "CO_1312/CO_65"   : "co131265",
    "CO_1413/CO_54"   : "co141354",
    "CO_1413/CO_65"   : "co141365",
    "OI_63+CII_158/FIR"     : "fir",
    "OI_145+CII_158/FIR"  : "firoi145",
  "SIII_Z1/FEII_Z1"  : "siii35feii26z1",
    "SIII_Z3/FEII_Z3"  : "siii35feii26z3",
    "H200S1_Z1/H200S0_Z1" : "h200s1s0z1",
    "H200S1_Z3/H200S0_Z3" : "h200s1s0z3",
    "H200S2_Z1/H200S0_Z1" : "h200s2s0z1",
    "H200S2_Z3/H200S0_Z3" : "h200s2s0z3",
    "H200S2_Z1/H200S1_Z1" : "h200s2s1z1",
    "H200S2_Z3/H200S1_Z3" : "h200s2s1z3",
    "H200S3_Z1/H200S1_Z1" : "h200s3s1z1",
    "H200S3_Z3/H200S1_Z3" : "h200s3s1z3",
    "H200S1_Z1/SIII_Z1" : "h200s1siiiz1",
    "H200S1_Z3/SIII_Z3" : "h200s1siiiz3",
    "H200S2_Z1/SIII_Z1" : "h200s2siiiz1",
    "H200S2_Z3/SIII_Z3" : "h200s2siiiz3",
    "H264Q1_Z1/H210S1_Z1" : "h264q110s1z1",
    "H264Q1_Z3/H210S1_Z3" : "h264q110s1z3"
}
a = list()
b = list()
for r in ratiodict:
    nd = r.split("/")
    if ("Z3" in r):
        z=3
    else:
        z=1
    b.append((nd[0],nd[1],r,ratiodict[r]+"web",z))
    a.append(ModelRatioData(nd[0],nd[1],r,ratiodict[r],z))
t = Table(rows=b,names=("numerator","denominator","label","filename","z"))
t.add_index("label")
t.write("current_models.tab",format="ascii.ipac",overwrite=True)

df = t.to_pandas()

In [406]:
m1 = Measurement(data=30.,uncertainty = StdDevUncertainty(5.),identifier="OI_145")
m2 = Measurement(data=10.,uncertainty = StdDevUncertainty(10.),identifier="CI_370")
m3 = Measurement(data=500.,uncertainty = StdDevUncertainty(50.),identifier="CO_10")
m4 = Measurement(data=100.,uncertainty = StdDevUncertainty(10.),identifier="CII_158")
m = [m1.id,m2.id,m3.id,m4.id]
md = dict()
for t in [m1,m2,m3,m4]:
    md[t.id] = t
print(md)
m1.__dict__

{'OI_145': Measurement(30.), 'CI_370': Measurement(10.), 'CO_10': Measurement(500.), 'CII_158': Measurement(100.)}


{'_identifier': 'OI_145',
 '_data': array(30.),
 '_mask': False,
 '_wcs': None,
 '_meta': OrderedDict(),
 '_unit': None,
 '_uncertainty': StdDevUncertainty(5.0),
 '_flags': None}

<b>Ok, so this error went poof.  After spending much of a day chasing this down, it has disappeared...</b>
*I don't really understand iterators.  Using find_ratio_elements in a loop causes an error if one tried to divide a Measurement indexed by numerator by a Measurement indexed by denominator. Example below. The error has to do with "no parent data" for the uncertainty.  It only happens with an iterator and not a list (get_ratio_elements).* 

In [407]:
p = PDRutils("current_models.tab")
m = pd.Series([m1.id,m2.id,m3.id,m4.id])
print(p.ratiocount(m))
k=list()
kk=list()

# this loop used to raise an exception about no parent data for uncertainty
for i in p.find_ratio_elements(m):
    _m1 = md[x[xx]["numerator"]]
    _m2 = md[x[xx]["denominator"]]
    _m3=_m1/_m2
    k.append(_m3)
    kk.append((_m1,_m2))

for pp in kk:
    f=pp[0]/pp[1]
    ff=pp[0]*pp[1]
    print(f.id,ff.id)
    print(f.uncertainty)

2
OI_145/CII_158 OI_145*CII_158
StdDevUncertainty(0.05830952)
OI_145/CII_158 OI_145*CII_158
StdDevUncertainty(0.05830952)


In [400]:
p = PDRutils("current_models.tab")
m = pd.Series([m1.id,m2.id,m3.id,m4.id])
p.ratiocount(m)
x = p.get_ratio_elements(m)
print(x)
#k = list()
for xx in range(len(x)):
   _m1 = md[x[xx]["numerator"]]
   _m2 = md[x[xx]["denominator"]]
   m3=_m1/_m2
   print("u=",m3.uncertainty)
    #k.append((_m1.__class__(_m1),m2.__class__(_m2)))\
#    k.append((_m1,_m2))
    
#for kk in k:
#    print(kk[0]/kk[1])
   #print(ratio,ratio.id)




[{'numerator': 'OI_145', 'denominator': 'CII_158'}]
100.0
0.3
u= StdDevUncertainty(0.05830952)


In [394]:
p.read_ccd(m,unit='flx')
print(p._fitsfiles)

{'o145ciiweb.fits': CCDData([[9.65481e-04, 5.90264e-04, 3.60887e-04, 2.20000e-04, 1.33728e-04,
          8.23615e-05, 5.49323e-05, 4.30108e-05, 4.32950e-05, 5.82418e-05,
          9.50413e-05, 1.61685e-04, 2.50895e-04, 3.09195e-04, 2.66667e-04,
          1.40659e-04, 4.06863e-05, 6.32312e-06, 8.26549e-07, 1.59893e-07,
          8.05774e-08, 1.58333e-07, 9.67980e-07, 1.26630e-05, 2.41794e-04],
         [1.80556e-03, 1.08974e-03, 6.56626e-04, 3.98824e-04, 2.45062e-04,
          1.53846e-04, 1.06838e-04, 8.64865e-05, 9.03090e-05, 1.26384e-04,
          2.17172e-04, 4.09774e-04, 7.46835e-04, 1.16875e-03, 1.36336e-03,
          1.01818e-03, 4.23488e-04, 9.36364e-05, 1.62915e-05, 3.66038e-06,
          1.75316e-06, 2.57353e-06, 9.66038e-06, 6.81363e-05, 6.55172e-04],
         [3.33803e-03, 1.98770e-03, 1.19343e-03, 7.23906e-04, 4.47368e-04,
          2.88235e-04, 2.05405e-04, 1.73991e-04, 1.87766e-04, 2.70513e-04,
          4.87903e-04, 9.94536e-04, 2.08609e-03, 4.02899e-03, 6.19632e-03,
   

In [395]:
q=m1/m2 
#print(((m1.flux*m2.flux)**2 * ((m1.uncertainty.array/m1.flux)**2 + (m2.uncertainty.array/m2.flux)**2))**0.5)
print(q,q.uncertainty,q.id)

10.0
3.0
OI_145 CI_370
3.0 StdDevUncertainty(3.04138127) OI_145/CI_370


In [396]:
q.id

'OI_145/CI_370'